## Translation from English to French

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00


In [16]:
from datasets import load_dataset

raw_dataset = load_dataset("kde4", lang1='en', lang2 = 'hi')

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for kde4 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/kde4
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [17]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 97227
    })
})

In [18]:
split_dataset = raw_dataset["train"].train_test_split(train_size=0.9, seed=20)
split_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 87504
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 9723
    })
})

In [19]:
split_dataset["validation"] = split_dataset.pop("test")

In [20]:
split_dataset['train'][0]

{'id': '14666',
 'translation': {'en': 'This button saves all your changes and exits the program.',
  'hi': 'यह बटन आपके सभी परिवर्तनों को सहेजता है तथा प्रोग्राम को बाहर कर देता है.'}}

In [7]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.9 MB/s eta 0:00:00


In [9]:
from transformers import pipeline

model_checkpoint = "Helsinki-NLP/opus-mt-en-hi"
translator = pipeline("translation", model=model_checkpoint)
translator("Default to expanded threads")

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/306M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


[{'translation_text': 'सभी लड़ी फैलाएँ (A)'}]

In [13]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")

In [22]:
split_dataset["train"][0]["translation"]

{'en': 'This button saves all your changes and exits the program.',
 'hi': 'यह बटन आपके सभी परिवर्तनों को सहेजता है तथा प्रोग्राम को बाहर कर देता है.'}

In [23]:
en_sentence = split_dataset["train"][1]["translation"]["en"]
hi_sentence = split_dataset["train"][1]["translation"]["hi"]

inputs = tokenizer(en_sentence, text_target=hi_sentence)
inputs

{'input_ids': [2866, 16910, 0], 'attention_mask': [1, 1, 1], 'labels': [8161, 10238, 0]}

In [25]:
max_length = 128

def preprocess_function(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["hi"] for ex in examples["translation"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs

In [27]:
tokenized_datasets = split_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=split_dataset["train"].column_names,
)

Map:   0%|          | 0/87504 [00:00<?, ? examples/s]

Map:   0%|          | 0/9723 [00:00<?, ? examples/s]

## Fine-tuning using Trainer API


In [37]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [38]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [39]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])

batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])